In [1]:
%run PilotForestAreas.ipynb

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [2]:
import param as pm
import panel as pn
import pandas as pd
import datetime as dt
pn.extension()

In [6]:
f1.sale_value()

3118225.187825

In [7]:
f1.forested_area_hectares

7478

In [8]:
f1.acres_protected()

18470.66

In [9]:
f1.tons_carbon_per_year

21867.58

In [10]:
class Carbon(pm.Parameterized):
    initial_carbon_price = pm.Number(29)
    carbon_price_appreciation = pm.Number(0.15, bounds=(0,1), step=0.01)
    stewardship_budget_increase = pm.Number(0.075, bounds=(0,1), step=0.01)
    tax_rate = pm.Number(0.2, bounds=(0,1), step=0.01)
    centree_admin_commission = pm.Number(0.15, bounds=(0,1), step=0.01)
    pfa_commission = pm.Number(0.425, bounds=(0,1), step=0.01)
    land_title_holders_commission = pm.Number(0.425, bounds=(0,1), step=0.01)
    below_soil_tonnage_factor = pm.Number(1, bounds=(0,2), step=0.01)
    
    def compute(self, forest_area: Forest):
        periods = 10
        freq = 'Y'
        
        df = pd.DataFrame(index=pd.date_range(dt.datetime.now(), periods=periods, freq=freq))
        df['Carbon Price'] = [self.initial_carbon_price * (1+self.carbon_price_appreciation)**x for x in range(periods)]

        df['Gross Revenue'] = df['Carbon Price'] * forest_area.tons_carbon_per_year * (1 + self.below_soil_tonnage_factor)

        df['Net Revenue'] = df['Gross Revenue'] * (1 - self.tax_rate)

        df['Stewardship Budget'] = [forest_area.forested_area_hectares * forest_area.stewardship_rate_per_hectare * (1+self.stewardship_budget_increase)**x for x in range(periods)]

        df['Gross Profit'] = df['Net Revenue'] - df['Stewardship Budget']

        df['Centree (Admin/Commission)'] = df['Gross Profit'] * self.centree_admin_commission

        df['PFA Commission'] = df['Gross Profit'] * self.pfa_commission
        df['Land Title Holders Commission'] = df['Gross Profit'] * self.land_title_holders_commission

        df['PFA Holder Return as Percentage of Initial Cost'] = df['PFA Commission'] / forest_area.sale_value()

        return df

In [12]:
Carbon(below_soil_tonnage_factor=0).compute(f1)

,Carbon Price,Gross Revenue,Net Revenue,Stewardship Budget,Gross Profit,Centree (Admin/Commission),PFA Commission,Land Title Holders Commission,PFA Holder Return as Percentage of Initial Cost
2023-12-31 20:44:37.653745,29.000000,6.341598e+05,5.073279e+05,209384.000000,2.979439e+05,44691.578400,126626.138800,126626.138800,0.040608
2024-12-31 20:44:37.653745,33.350000,7.292838e+05,5.834270e+05,225087.800000,3.583392e+05,53750.885160,152294.174620,152294.174620,0.048840
2025-12-31 20:44:37.653745,38.352500,8.386764e+05,6.709411e+05,241969.385000,4.289717e+05,64345.755684,182312.974438,182312.974438,0.058467
2026-12-31 20:44:37.653745,44.105375,9.644778e+05,7.715823e+05,260117.088875,5.114652e+05,76719.774618,217372.694751,217372.694751,0.069710
2027-12-31 20:44:37.653745,50.721181,1.109149e+06,8.873196e+05,279625.870541,6.076937e+05,91154.058060,258269.831171,258269.831171,0.082826
2028-12-31 20:44:37.653745,58.329358,1.275522e+06,1.020418e+06,300597.810831,7.198197e+05,107972.957813,305923.380470,305923.380470,0.098108
2029-12-31 20:44:37.653745,67.078762,1.466850e+06,1.173480e+06,323142.646644,8.503375e+05,127550.626857,361393.442761,361393.442761,0.115897
2030-12-31 20:44:37.653745,77.140577,1.686878e+06,1.349502e+06,347378.345142,1.002124e+06,150318.575660,425902.631037,425902.631037,0.136585
2031-12-31 20:44:37.653745,88.711663,1.939909e+06,1.551928e+06,373431.721027,1.178496e+06,176774.368392,500860.710444,500860.710444,0.160624
2032-12-31 20:44:37.653745,102.018412,2.230896e+06,1.784717e+06,401439.100104,1.383278e+06,207491.630512,587892.953118,587892.953118,0.188534


In [13]:
Carbon(below_soil_tonnage_factor=1).compute(f1)

,Carbon Price,Gross Revenue,Net Revenue,Stewardship Budget,Gross Profit,Centree (Admin/Commission),PFA Commission,Land Title Holders Commission,PFA Holder Return as Percentage of Initial Cost
2023-12-31 20:44:42.599169,29.000000,1.268320e+06,1.014656e+06,209384.000000,8.052717e+05,120790.756800,3.422405e+05,3.422405e+05,0.109755
2024-12-31 20:44:42.599169,33.350000,1.458568e+06,1.166854e+06,225087.800000,9.417663e+05,141264.940320,4.002507e+05,4.002507e+05,0.128358
2025-12-31 20:44:42.599169,38.352500,1.677353e+06,1.341882e+06,241969.385000,1.099913e+06,164986.919118,4.674629e+05,4.674629e+05,0.149913
2026-12-31 20:44:42.599169,44.105375,1.928956e+06,1.543165e+06,260117.088875,1.283047e+06,192457.112567,5.452952e+05,5.452952e+05,0.174874
2027-12-31 20:44:42.599169,50.721181,2.218299e+06,1.774639e+06,279625.870541,1.495013e+06,224251.996702,6.353807e+05,6.353807e+05,0.203764
2028-12-31 20:44:42.599169,58.329358,2.551044e+06,2.040835e+06,300597.810831,1.740237e+06,261035.587251,7.396008e+05,7.396008e+05,0.237186
2029-12-31 20:44:42.599169,67.078762,2.933700e+06,2.346960e+06,323142.646644,2.023818e+06,303572.650710,8.601225e+05,8.601225e+05,0.275837
2030-12-31 20:44:42.599169,77.140577,3.373755e+06,2.699004e+06,347378.345142,2.351626e+06,352743.903091,9.994411e+05,9.994411e+05,0.320516
2031-12-31 20:44:42.599169,88.711663,3.879819e+06,3.103855e+06,373431.721027,2.730423e+06,409563.494938,1.160430e+06,1.160430e+06,0.372144
2032-12-31 20:44:42.599169,102.018412,4.461792e+06,3.569433e+06,401439.100104,3.167994e+06,475199.126040,1.346398e+06,1.346398e+06,0.431783
